In [ ]:
# @title Setup: install, imports, mount Drive, shared config, genus-30 cuts
!pip install -q torch numpy mpmath tqdm

import os, time, math, json, numpy as np, torch, mpmath as mp
from tqdm import tqdm
from google.colab import drive

# -----------------------------
# Drive & save directory
# -----------------------------
DRIVE_FOLDER = "AJ_Tables_g30"   # << change if you like
drive.mount('/content/drive', force_remount=True)
SAVE_DIR = f"/content/drive/MyDrive/{DRIVE_FOLDER}"
os.makedirs(SAVE_DIR, exist_ok=True)

# File paths (ω and I saved separately, resume-safe)
OMEGAS_PATH    = os.path.join(SAVE_DIR, "aj_omegas_genus30.pt")
INTEGRALS_PATH = os.path.join(SAVE_DIR, "aj_integrals_genus30.pt")
print("Will save to:\n ", OMEGAS_PATH, "\n ", INTEGRALS_PATH)

# -----------------------------
# Genus & grid/precision
# -----------------------------
genus = 30                           # <<< g
H = W = 96                            # start modest (e.g., 96 or 112); raise later (140–180)
r_min, r_max = -6.0, 6.0
i_min, i_max = -6.0, 6.0
grid_r = np.linspace(r_min, r_max, W).astype(np.float64)
grid_i = np.linspace(i_min, i_max, H).astype(np.float64)

mp.mp.dps = 50                        # precision; raise to 70–90 if needed
base_point = complex(r_min - 2.0, i_min - 2.0)

# -----------------------------
# Helper: atomic save
# -----------------------------
def atomic_torch_save(obj, path):
    tmp = path + ".tmp"
    torch.save(obj, tmp)
    os.replace(tmp, path)

# -----------------------------
# Genus-30 cuts: robust, non-overlapping construction
# -----------------------------
def make_hyperelliptic_cuts(g, radius=4.0, jitter=0.25, seed=123):
    """
    Returns g+1 cuts [(a0,a1), ..., (a_{g}, a_{g}')] for a hyperelliptic curve
    with 2g+2 branch points. We place 2g+2 points on a perturbed circle and
    pair neighbors to form short segments; this avoids overlaps and keeps
    separation reasonable.

    radius: circle radius for base placement
    jitter: random radial jitter (<= ~0.3 recommended)
    """
    rng = np.random.RandomState(seed)
    m = 2*g + 2
    thetas = np.linspace(0, 2*np.pi, m, endpoint=False)
    # random angular shuffle to avoid long aligned runs
    rng.shuffle(thetas)
    radii = radius * (1.0 + jitter * (rng.rand(m) - 0.5))
    pts = radii * np.exp(1j * thetas)   # complex on approximate circle
    # ensure within grid box
    scale = max((pts.real.max()-pts.real.min())/(r_max-r_min+1e-6),
                (pts.imag.max()-pts.imag.min())/(i_max-i_min+1e-6))
    if scale > 0.85:
        pts = pts / (scale/0.85)

    # pair each point to its nearest neighbor not yet paired, to form short cuts
    remaining = list(range(m))
    cuts = []
    while remaining:
        i = remaining.pop(0)
        pi = pts[i]
        # find nearest j still remaining
        dists = [(j, abs(pi - pts[j])) for j in remaining]
        j = min(dists, key=lambda t: t[1])[0]
        remaining.remove(j)
        # shorten segment slightly so integral endpoints are not exact branch points
        a, b = pi, pts[j]
        mid = 0.5*(a+b)
        a = a + 0.05*(a - mid)
        b = b + 0.05*(b - mid)
        cuts.append((complex(a), complex(b)))
    # We now have g+1 cuts (since m = 2g+2 and we paired them). If pairing produced > g+1
    # due to rounding, trim to g+1 by dropping the longest segments (unlikely).
    if len(cuts) > g+1:
        cuts.sort(key=lambda ab: -abs(ab[0]-ab[1]))
        cuts = cuts[:g+1]
    assert len(cuts) == g+1, f"Expected {g+1} cuts, got {len(cuts)}."
    return cuts

branch_cuts = make_hyperelliptic_cuts(genus)
branch_pts  = [a for ab in branch_cuts for a in ab]
print(f"genus={genus} → cuts={len(branch_cuts)} ; total branch points={len(branch_pts)}")

# -----------------------------
# Differentials ω_k and line integral
# -----------------------------
def make_omega(branch_points):
    pts = list(branch_points)
    def omega_k(k, t):
        # ω_k(t) = t^k / sqrt(prod (t - a_i))
        prod = mp.mpf(1)
        for a in pts:
            prod *= (t - a)
        return t**k / mp.sqrt(prod)
    return omega_k

omega = make_omega(branch_pts)

def integrate_omega(k, z):
    # Segment [base_point -> z] with tiny endpoint nudge
    try:
        return mp.quad(lambda t: omega(k, t), [base_point, z])
    except Exception:
        eps = 1e-12 + 1e-12j
        return mp.quad(lambda t: omega(k, t), [base_point, z + eps])

# -----------------------------
# Common metadata for files
# -----------------------------
COMMON_META = {
    "genus": genus,
    "branch_cuts": branch_cuts,
    "branch_pts": np.array(branch_pts, dtype=np.complex128),
    "grid_r": grid_r, "grid_i": grid_i,
    "meta": {"mp_dps": mp.mp.dps, "base_point": base_point,
             "grid_shape": (H, W), "ranges": (r_min, r_max, i_min, i_max)}
}

def ensure_config_compatible(payload):
    assert int(payload["genus"]) == genus
    assert payload["meta"]["grid_shape"] == (H, W)
    assert np.allclose(payload["grid_r"], grid_r)
    assert np.allclose(payload["grid_i"], grid_i)


Mounted at /content/drive
Will save to:
  /content/drive/MyDrive/AJ_Tables_g30/aj_omegas_genus30.pt 
  /content/drive/MyDrive/AJ_Tables_g30/aj_integrals_genus30.pt
genus=30 → cuts=31 ; total branch points=62


In [ ]:
import os
import datetime

if 'INTEGRALS_PATH' in globals() and os.path.exists(INTEGRALS_PATH):
    mtime = os.path.getmtime(INTEGRALS_PATH)
    print(f"Last modification time of {os.path.basename(INTEGRALS_PATH)}: {datetime.datetime.fromtimestamp(mtime)}")
else:
    print(f"File not found or INTEGRALS_PATH not defined: {INTEGRALS_PATH if 'INTEGRALS_PATH' in globals() else 'Not Defined'}")


Last modification time of aj_integrals_genus30.pt: 2025-11-07 22:34:35


In [ ]:
# @title Check/lock branch cuts against saved files (safe to run right after Cell 0)
import os, numpy as np, torch

# Toggle: if mismatch is found, automatically replace in-memory cuts/points with the saved ones.
AUTO_ADOPT_SAVED_CUTS = False   # set True to auto-fix

# Helper: pick a saved file to compare against (prefer integrals, else omegas)
def pick_saved_path():
    cand = []
    if 'INTEGRALS_PATH' in globals() and os.path.exists(INTEGRALS_PATH):
        cand.append(INTEGRALS_PATH)
    if 'OMEGAS_PATH' in globals() and os.path.exists(OMEGAS_PATH):
        cand.append(OMEGAS_PATH)
    if not cand:
        return None
    # Prefer the file with the newest mtime
    cand.sort(key=lambda p: os.path.getmtime(p), reverse=True)
    return cand[0]

def np_c128_from_list(lst):
    return np.array([complex(z) for z in lst], dtype=np.complex128)

# Sanity: required globals from Cell 0
for name in ["genus", "grid_r", "grid_i", "branch_cuts", "branch_pts"]:
    if name not in globals():
        raise RuntimeError(f"Expected `{name}` from Cell 0 to be defined.")

saved_path = pick_saved_path()
if saved_path is None:
    print("No saved ω/Ι file found in Drive — looks like a fresh run; nothing to compare.")
else:
    pkg = torch.load(saved_path, map_location="cpu", weights_only=False)
    saved_genus = int(pkg.get("genus"))
    saved_grid_r = np.array(pkg.get("grid_r"))
    saved_grid_i = np.array(pkg.get("grid_i"))
    saved_pts = np.array(pkg.get("branch_pts", []), dtype=np.complex128)
    saved_cuts = pkg.get("branch_cuts", None)

    curr_pts = np_c128_from_list(branch_pts)

    print(f"Comparing current session to: {os.path.basename(saved_path)}")
    ok_genus = (saved_genus == genus)
    ok_grid  = np.allclose(saved_grid_r, grid_r) and np.allclose(saved_grid_i, grid_i)
    ok_len   = (saved_pts.shape == curr_pts.shape)
    ok_pts   = ok_len and np.allclose(saved_pts.real, curr_pts.real) and np.allclose(saved_pts.imag, curr_pts.imag)

    print(f"  genus match : {ok_genus} (saved={saved_genus}, current={genus})")
    print(f"  grid match  : {ok_grid}")
    if ok_len:
        max_dev = float(np.max(np.abs(saved_pts - curr_pts))) if saved_pts.size else 0.0
        print(f"  branch pts  : {'MATCH' if ok_pts else 'MISMATCH'} (max |Δ| = {max_dev:.3e})")
    else:
        print(f"  branch pts  : count differs (saved {saved_pts.size}, current {curr_pts.size})")

    if ok_genus and ok_grid and ok_pts:
        print("\n✅ SAFE to resume: cuts/points match what’s in Drive.")
    else:
        print("\n❌ MISMATCH detected — do NOT resume building tables with different cuts.")
        if AUTO_ADOPT_SAVED_CUTS and (saved_cuts is not None):
            # Adopt saved geometry into this session to ensure consistency
            branch_cuts = saved_cuts
            branch_pts  = [a for ab in branch_cuts for a in ab]
            # Rebuild the differential generator with the adopted branch points
            if 'make_omega' in globals():
                omega = make_omega(branch_pts)  # rebind omega to the saved geometry
            print("→ Adopted the saved branch cuts/points into memory. You can now safely resume.")
        else:
            print("Tip: set AUTO_ADOPT_SAVED_CUTS=True at the top of this cell to adopt the saved cuts.")


Comparing current session to: aj_integrals_genus30.pt
  genus match : True (saved=30, current=30)
  grid match  : True
  branch pts  : MATCH (max |Δ| = 0.000e+00)

✅ SAFE to resume: cuts/points match what’s in Drive.


In [ ]:
# @title Build ω (differentials) for g=30 → Drive (resume-safe)
import torch

# Initialize or resume
if os.path.exists(OMEGAS_PATH):
    pkg = torch.load(OMEGAS_PATH, map_location='cpu', weights_only=False)
    ensure_config_compatible(pkg)
    Om_plus = pkg["omega_plus"]                  # (g, H, W) complex
    progress = pkg.get("progress", {"iy_done": [0]*genus, "k_done": 0})
    print("Resuming ω from Drive.")
else:
    Om_plus = torch.zeros(genus, H, W, dtype=torch.cfloat)
    progress = {"iy_done": [0]*genus, "k_done": 0}
    print("Starting fresh ω build.")

SAVE_EVERY_N_ROWS = 1  # safest; can raise to 2–4 to reduce I/O

t0 = time.time()
for k in range(progress["k_done"], genus):
    start_iy = int(progress["iy_done"][k])
    if start_iy >= H:
        print(f"ω[{k}] already complete ({H} rows).")
        progress["k_done"] = k+1
        continue

    print(f"Building ω[{k}] rows {start_iy}..{H-1}")
    rows_since_save = 0
    for iy in tqdm(range(start_iy, H), desc=f"omega[{k}]"):
        y = grid_i[iy]
        for ix in range(W):
            x = grid_r[ix]
            z = complex(x, y)
            Om_plus[k, iy, ix] = complex(omega(k, z))
        progress["iy_done"][k] = iy + 1
        rows_since_save += 1

        if rows_since_save >= SAVE_EVERY_N_ROWS:
            payload = {**COMMON_META, "omega_plus": Om_plus, "progress": progress}
            atomic_torch_save(payload, OMEGAS_PATH)
            rows_since_save = 0

    # finalize this k
    progress["k_done"] = k+1
    payload = {**COMMON_META, "omega_plus": Om_plus, "progress": progress}
    atomic_torch_save(payload, OMEGAS_PATH)

t1 = time.time()
print(f"ω table saved to {OMEGAS_PATH}  | elapsed {t1 - t0:.1f}s")


Starting fresh ω build.
Building ω[0] rows 0..95


omega[0]: 100%|██████████| 96/96 [00:06<00:00, 14.46it/s]


Building ω[1] rows 0..95


omega[1]: 100%|██████████| 96/96 [00:06<00:00, 14.58it/s]


Building ω[2] rows 0..95


...

Building ω[28] rows 0..95


omega[28]: 100%|██████████| 96/96 [00:07<00:00, 13.36it/s]


Building ω[29] rows 0..95


omega[29]: 100%|██████████| 96/96 [00:07<00:00, 13.19it/s]

ω table saved to /content/drive/MyDrive/AJ_Tables_g30/aj_omegas_genus30.pt  | elapsed 207.0s


In [ ]:
# @title Build I (integrals) for g=30 → Drive (resume-safe)
import torch

# Initialize or resume
if os.path.exists(INTEGRALS_PATH):
    pkg = torch.load(INTEGRALS_PATH, map_location='cpu', weights_only=False)
    ensure_config_compatible(pkg)
    I_plus = pkg["I_plus"]                       # (g, H, W) complex
    progress = pkg.get("progress", {"iy_done": [0]*genus, "k_done": 0})
    print("Resuming integrals from Drive.")
else:
    I_plus = torch.zeros(genus, H, W, dtype=torch.cfloat)
    progress = {"iy_done": [0]*genus, "k_done": 0}
    print("Starting fresh integrals build.")

SAVE_EVERY_N_ROWS = 1  # safest; can raise to 2–4 later

t0 = time.time()
for k in range(progress["k_done"], genus):
    start_iy = int(progress["iy_done"][k])
    if start_iy >= H:
        print(f"I[{k}] already complete ({H} rows).")
        progress["k_done"] = k+1
        continue

    print(f"Building I[{k}] rows {start_iy}..{H-1}")
    rows_since_save = 0
    for iy in tqdm(range(start_iy, H), desc=f"I[{k}]   "):
        y = grid_i[iy]
        for ix in range(W):
            x = grid_r[ix]
            z = complex(x, y)
            I_plus[k, iy, ix] = complex(integrate_omega(k, z))
        progress["iy_done"][k] = iy + 1
        rows_since_save += 1

        if rows_since_save >= SAVE_EVERY_N_ROWS:
            payload = {**COMMON_META, "I_plus": I_plus, "progress": progress}
            atomic_torch_save(payload, INTEGRALS_PATH)
            rows_since_save = 0

    # finalize this k
    progress["k_done"] = k+1
    payload = {**COMMON_META, "I_plus": I_plus, "progress": progress}
    atomic_torch_save(payload, INTEGRALS_PATH)

t1 = time.time()
print(f"I table saved to {INTEGRALS_PATH}  | elapsed {t1 - t0:.1f}s")


Resuming integrals from Drive.
Building I[26] rows 86..95


I[26]   : 100%|██████████| 10/10 [29:34<00:00, 177.43s/it]


Building I[27] rows 0..95


I[27]   : 100%|██████████| 96/96 [4:41:34<00:00, 175.99s/it]


Building I[28] rows 0..95


I[28]   : 100%|██████████| 96/96 [4:42:49<00:00, 176.76s/it]


Building I[29] rows 0..95


I[29]   : 100%|██████████| 96/96 [4:40:40<00:00, 175.43s/it]

I table saved to /content/drive/MyDrive/AJ_Tables_g30/aj_integrals_genus30.pt  | elapsed 52479.3s
